In [8]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_colwidth', None)
from transformers import pipeline
import markdown

In [9]:
data = pd.read_csv('data.csv')
data

,Context,Response,Sentiment
0,hi,Hello there. Tell me how are you feeling today?,positive
1,hi,Hi there. What brings you here today?,positive
2,hi,Hi there. How are you feeling today?,positive
3,hi,Great to see you. How do you feel currently?,positive
4,hi,Hello there. Glad to see you're back. What's going on in your world right now?,positive
...,...,...,...
18254,my grandson's step-mother sends him to school with a pink barbie backpack as a form of punishment.,"Absolutely not! It is never in a child's best interest to use humiliation as punishment. This can lead to issues in the relationship between parent and child as well as the child's social relationships. Kids have a hard enough time. To then go and cause them embarrassment is unconscionable. The entire job of a parent is to prepare their child to become a successful, healthy adult. You do this by using healthy ways to teach a child when they make a mistake. Discipline shouldn't be seen as punishment. It should be seen as a way to teach a lesson. What lesson could this possibility be teaching him? Absolutely nothing. Meanwhile, she is most likely damaging her relationship with him and causing great distress in his school day and peer interactions. She should be one of the people he can come to when he's hurting; not someone who causes him pain.",negative
18255,my boyfriend is in recovery from drug addiction. we recently got into a fight and he has become very distant. i don't know what to do to fix the relationship.,"I'm sorry you have tension between you and your bf.A relationship means two people who relate, right?! If only one person does all the work to change their approach, what they expect, what they offer the other, then this becomes the new problem to solve.If you've already done a significant amount of reflection and change in the way you relate to him, then the next step may be to be patient as long as possible while he decides what to change about his part in the relationship toward you.Substance abuse recovery requires profound effort and reflection. If it is successful then the result will show very different qualities in your bf than the ones you know currently.He may be so absorbed in the recovery that he can only concentrate on this. Of course if the fights persist and you have been patient for what feels too long, then there are different questions to address.",negative
18256,"the birth mother attempted suicide several times while pregnant. the adopted mother was terminally ill upon adopting the baby and died when the child was just over one year old. the adopted father then remarried to a physically, psychologically, and emotionally abusive woman. the child was placed in foster care at 11 years old. what might be the long term effects on an adult with this childhood history?","The true answer is, ""no one can really say with certainty"".The variables are the way this child absorbs and adjusts to these significant changes in their life. All anyone can do is guess at this point and there's no good reason to guess.The only general certainty is that the adult whom this child becomes will have had a profound encounter with the biggest types of human losses a child can go through.Some people become great teachers, therapists and philosophers who have this background. Some give up on life and hide away from others.The best anyone could do who knows this child is to offer love bc this is the greatest guarantee to show there are good people on this earth.",negative
18257,"i think adult life is making him depressed and we often sleep in on weekends untill 1 or 2 pm. we just eat, smoke weed, watch movies, and he'll go on his phone all day. he doesn't seem motivated to do much and he's often angry. i have no clue how to help him take his mind off the negative, or to distract him into a different light. how do i help him?",How do you help yourself to believe you require more than what he offers to you?What do you get from this relationship which feels satisfying?To

In [10]:
data = data.drop_duplicates(subset=['Context'], ignore_index=True)
data

,Context,Response,Sentiment
0,hi,Hello there. Tell me how are you feeling today?,positive
1,hey,Hello there. Tell me how are you feeling today?,positive
2,is anyone there?,Hello there. Tell me how are you feeling today?,negative
3,hi there,Hello there. Tell me how are you feeling today?,positive
4,hello,Hello there. Tell me how are you feeling today?,positive
...,...,...,...
7879,"i have been diagnosed with adhd and experienced manic depression episodes. i have problems with anger management. apparently, i also have an odd, bipolar and split personality. how can i be truly happy?","The ADHD and manic depressive episodes are terms other people told you that you have.You don't have to accept these words to describe who you are.A lot of mental health in our country is about telling people what's wrong with them and giving pills for these so called conditions, instead of helping a person know more about themselves in order to make good decisions.You can be happy because nothing from the outside, such as the names people have used to tell you who you are, can interfere with your own wish to be happy.Others can slow you down because of the self-doubt and hurt feelings. No one can take happiness from you, only they can make it harder to find and hold onto.Anger management only works temporarily because it is a surface approach.Think of the reason you are angry. This will be better to know so you will be able to address it.Then you will not have anger to manage.",negative
7880,"tonight, my husband seemed to put our son down through an incorrect approach. his approach was perfectly wrong. i found myself defending my son. i told him that what he did was not the way to encourage our son. instead of watching my husband’s harsh behavior i decided to sit beside my son and, together, we worked on his science assignment in order to encourage him to study. my husband got upset and he was swearing at me. he threatened me. i knew that he was not joking. his words scared me a lot!","I have had these many cases, but in situations like this.... If it does get out of hand, the police do need to be involved. Sit down and talk to your husband when he is calm and collective.",negative
7881,"he's been losing feelings and he doesn't know why. i love him very very much. he sometimes thinks i'm obsessed when i'm not at all. i give him his space and i make sure he's okay but sometimes i think if me and him saw each other more it would be better? i truly want me and him to get better, it's kind of hard not to stress about it, because the love of my life is losing feelings which is a sad feeling. he's a great guy! i just don't know why he's been losing feelings towards me. he's starting to put less effort in talking to me. at this point i'll do anything to just make us better as a couple. i tried talking to him but he doesn't like talking about it much. advice on what to do?","I'm willing to bet that this isn't what you are hoping to hear, but I'd suggest giving him space. Ooph, that's a tough one, right?! I know. But here's the thing, when you keep trying to process and talk it out with him you keep pushing him away. He needs to feel like a solid and whole person (as you do too) to be able to be fully in your relationship. It's the work of being in relationship to learn this. I get that this is tough stuff. I wonder, outside of being with him, how do you soothe and calm yourself? That's the stuff for you to tune into and focus on right now.",negative
7882,my boyfriend can't get over my promiscuous past. he says he loves me but says the thought of my past is disgusting and a sexual distraction. he says he should have had more sex with different people. i don't know how to handle this.,"I am sorry to hear of your relationship struggles. One thing I wonder is did you volunteer the information about your past or is this something that he asked about? In the future, it is probably best to leave details out of conversations like this. I think it is common and natu

In [50]:

class Model:
    __transformer_embedding = None
    __transformer = None
    __data = None
    __classifier = None
    
    def __init__(self, data):
        self.__data = data
        self.__classifier = pipeline("sentiment-analysis")
        self.__transformer= SentenceTransformer('all-MiniLM-L6-v2')
        self.__transformer_embedding = self.__transformer.encode(data['Context']).tolist()

    def __get_sentimet(self,text):
        result = self.__classifier(text)[0]
        return result['label'].lower()
    def __get_response_transformer(self, user_input):
        user_embedding = self.__transformer.encode([user_input.lower()])
        similarities = cosine_similarity(user_embedding, self.__transformer_embedding).flatten()
        user_sentiment = self.__get_sentimet(user_input)
        sentiment_similarities = self.__data['Sentiment'].apply(lambda x: 1 if x == user_sentiment else 0)

        similarities = similarities * sentiment_similarities
        top_indices = similarities.argsort()[-5:][::-1] 
        max_index = similarities.argmax()

        if similarities[max_index] > 0.70:
            return self.__data.iloc[max_index]['Response']
        else:
            possible_questions = self.__data.iloc[top_indices]['Context'].to_list()
            return  "Sorry, I don't understand. Can you please rephrase your question? " "Or try one of the following questions:<br><ul>" + "".join(f"<li>{question}</li>" for question in possible_questions) + "</ul>"

    def get_response(self,user_input):
        response = self.__get_response_transformer(user_input)
        return response
        

In [51]:
model = Model(data)

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [52]:
model.get_response("i am not feeling okay, about my future")

"It's normal to feel anxious about the future, but it's important to manage those feelings before they become overwhelming. Let's discuss some relaxation techniques and explore any underlying issues that may be causing your anxiety."

In [53]:
import dill
with open('model.pkl', 'wb') as f:
    dill.dump(model, f)

In [54]:
import dill
model = dill.load(open('model.pkl', 'rb'))

In [55]:
model.get_response("i don't feel confident about my future")

"Anxiety about the future is common, but it's important to remember that you have the power to shape your own path. Let's work together to develop strategies to manage your anxiety and create a plan to achieve your goals."